<a href="https://colab.research.google.com/github/wrzoskamil/UMWF/blob/main/use_case2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
movies_df = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')
ratings_df = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')

In [ ]:
# Zadanie 1
# 1. Podaj liczbę filmów Sci-Fi
sci_fi_movies_count = movies_df[movies_df['genres'].str.contains('Sci-Fi')].shape[0]
print("Liczba filmów Sci-Fi:", sci_fi_movies_count)

Liczba filmów Sci-Fi: 980


In [ ]:
# 2. Pokaż rozkład ocen komedii z 2017
comedies_2017 = movies_df[(movies_df['genres'].str.contains('Comedy')) & (movies_df['title'].str.contains('\(2017\)'))]
comedies_2017_ratings = pd.merge(comedies_2017, ratings_df, on='movieId')
rating_distribution = comedies_2017_ratings['rating'].value_counts().sort_index()
print("Rozkład ocen komedii z 2017 roku:")
print(rating_distribution)

Rozkład ocen komedii z 2017 roku:
rating
0.5     2
1.0     2
1.5     5
2.0     5
2.5     6
3.0    12
3.5    17
4.0    24
4.5    10
5.0     5
Name: count, dtype: int64


In [ ]:
# 3. Podaj średnią ocen wszystkich filmów akcji oraz 3 filmy akcji najczęściej oceniane

action_movies = movies_df[movies_df['genres'].str.contains('Action')]
action_movies_ratings = pd.merge(action_movies, ratings_df, on='movieId')
# Średnia ocena filmów akcji
average_rating_action = action_movies_ratings['rating'].mean()
print("Średnia ocen filmów akcji:", average_rating_action)

# Trzy najczęściej oceniane filmy akcji
top_rated_action_movies_ids = action_movies_ratings['movieId'].value_counts().head(3).index
top_rated_action_movies = movies_df[movies_df['movieId'].isin(top_rated_action_movies_ids)]
print("Trzy najczęściej oceniane filmy akcji:")
for idx in top_rated_action_movies_ids:
    title = top_rated_action_movies[top_rated_action_movies['movieId'] == idx]['title'].values[0]
    count = action_movies_ratings[action_movies_ratings['movieId'] == idx]['rating'].count()
    print(f"{title}: {count} ocen")

Średnia ocen filmów akcji: 3.447984331646809
Trzy najczęściej oceniane filmy akcji:
Matrix, The (1999): 278 ocen
Star Wars: Episode IV - A New Hope (1977): 251 ocen
Jurassic Park (1993): 238 ocen


In [ ]:
!pip install scikit-surprise
from surprise import SVD, KNNBaseline, Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV

ratings = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv')
movies = pd.read_csv('https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv')

In [ ]:
# Zadanie 2 - System rokomendacyjny
# Przygotowanie danych do formatu, który można przekazać do Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# Definicja i walidacja modelu SVD
algo_svd = SVD()
cross_validate(algo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Definicja i walidacja modelu kNN Baseline
sim_options = {'name': 'pearson_baseline', 'user_based': True}
algo_knn_baseline = KNNBaseline(sim_options=sim_options)
cross_validate(algo_knn_baseline, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Hiperparametryzacja dla kNN Baseline używając GridSearch
param_grid = {'k': [2, 3, 4, 5, 6], 'sim_options': {'name': ['pearson_baseline'], 'user_based': [True, False]}}
gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# Wyniki hiperparametryzacji
print("Najlepsze parametry dla RMSE:", gs.best_params['rmse'])
print("Najlepsze wyniki dla RMSE:", gs.best_score['rmse'])

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8771  0.8807  0.8707  0.8661  0.8793  0.8748  0.0055  
MAE (testset)     0.6755  0.6763  0.6674  0.6653  0.6765  0.6722  0.0048  
Fit time          1.72    1.59    1.76    1.75    2.33    1.83    0.26    
Test time         0.13    0.14    0.37    0.12    0.15    0.18    0.09    
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing simil

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

In [ ]:
all_movie_ids = ratings['movieId'].unique()

# Utworzenie listy par
penguins_user_movie_pairs = [(user_id, penguins_id) for user_id in ratings['userId'].unique()]
hobbit_user_movie_pairs = [(user_id, hobbit_id) for user_id in ratings['userId'].unique()]

# Uzyskanie przewidywanych ocen dla wszystkich użytkowników za pomocą modelu SVD
penguins_svd_ratings = [(user_id, algo_svd.predict(user_id, penguins_id).est) for user_id, penguins_id in penguins_user_movie_pairs]
hobbit_svd_ratings = [(user_id, algo_svd.predict(user_id, hobbit_id).est) for user_id, hobbit_id in hobbit_user_movie_pairs]

# Sortowanie przewidywanych ocen malejąco i wybranie 10 najlepszych rekomendacji
penguins_svd_recommendations = sorted(penguins_svd_ratings, key=lambda x: x[1], reverse=True)[:10]
hobbit_svd_recommendations = sorted(hobbit_svd_ratings, key=lambda x: x[1], reverse=True)[:10]

print("Rekomendacje po obejrzeniu filmu 'Penguins of Madagascar' za pomocą modelu SVD:")
for user_id, rating in penguins_svd_recommendations:
    print(f"Użytkownik {user_id}: przewidywana ocena = {rating}")

print("\nRekomendacje po obejrzeniu filmu 'Hobbit: The Desolation of Smaug, The' za pomocą modelu SVD:")
for user_id, rating in hobbit_svd_recommendations:
    print(f"Użytkownik {user_id}: przewidywana ocena = {rating}")


Rekomendacje po obejrzeniu filmu 'Penguins of Madagascar' za pomocą modelu SVD:
Użytkownik 53: przewidywana ocena = 4.545122439806879
Użytkownik 523: przewidywana ocena = 4.4838094551045495
Użytkownik 171: przewidywana ocena = 4.429329625878524
Użytkownik 12: przewidywana ocena = 4.387622479903762
Użytkownik 43: przewidywana ocena = 4.38743057565517
Użytkownik 371: przewidywana ocena = 4.385899855872154
Użytkownik 1: przewidywana ocena = 4.372404141351187
Użytkownik 452: przewidywana ocena = 4.357590453145033
Użytkownik 93: przewidywana ocena = 4.34169531767337
Użytkownik 122: przewidywana ocena = 4.324491098291593

Rekomendacje po obejrzeniu filmu 'Hobbit: The Desolation of Smaug, The' za pomocą modelu SVD:
Użytkownik 53: przewidywana ocena = 4.548496990635414
Użytkownik 452: przewidywana ocena = 4.451758539742081
Użytkownik 586: przewidywana ocena = 4.4380217322358995
Użytkownik 12: przewidywana ocena = 4.433774069539916
Użytkownik 122: przewidywana ocena = 4.407734591091876
Użytkown

In [ ]:
# Wyświetlenie identyfikatorów filmów zawierających 'Penguins'
print("Identyfikatory filmów zawierających 'Penguins':")
print(movies[movies['title'].str.contains("Penguins", case=False, na=False)])

# Wyświetlenie identyfikatorów filmów zawierających 'Hobbit'
print("\nIdentyfikatory filmów zawierających 'Hobbit':")
print(movies[movies['title'].str.contains("Hobbit", case=False, na=False)])


Identyfikatory filmów zawierających 'Penguins':
      movieId                                              title  \
5932    34072  March of the Penguins (Marche de l'empereur, L...   
7629    87483                       Mr. Popper's Penguins (2011)   
8586   117368  The Madagascar Penguins in a Christmas Caper (...   
8599   117851                      Penguins of Madagascar (2014)   

                                   genres  
5932                          Documentary  
7629                               Comedy  
8586                     Animation|Comedy  
8599  Adventure|Animation|Children|Comedy  

Identyfikatory filmów zawierających 'Hobbit':
      movieId                                             title  \
8053    98809         Hobbit: An Unexpected Journey, The (2012)   
8296   106489       Hobbit: The Desolation of Smaug, The (2013)   
8617   118696  The Hobbit: The Battle of the Five Armies (2014)   

                      genres  
8053  Adventure|Fantasy|IMAX  
8296  Adventu

In [ ]:
# Mapowanie identyfikatorów filmów na ich tytuły
movie_titles = dict(zip(movies['movieId'], movies['title']))

# Identyfikatory filmów
penguins_ids = [34072, 87483, 117368, 117851]
hobbit_ids = [98809, 106489, 118696]

# Rekomendacje dla Penguins of Madagascar
print("Rekomendowane filmy po obejrzeniu 'Penguins of Madagascar':")
for movie_id in penguins_ids:
    if movie_id in movie_titles:
        print(movie_titles[movie_id])
    else:
        print(f"Brak informacji o filmie o identyfikatorze {movie_id}.")

# Rekomendacje dla Hobbit
print("\nRekomendowane filmy po obejrzeniu 'Hobbit: The Desolation of Smaug, The':")
for movie_id in hobbit_ids:
    if movie_id in movie_titles:
        print(movie_titles[movie_id])
    else:
        print(f"Brak informacji o filmie o identyfikatorze {movie_id}.")


Rekomendowane filmy po obejrzeniu 'Penguins of Madagascar':
March of the Penguins (Marche de l'empereur, La) (2005)
Mr. Popper's Penguins (2011)
The Madagascar Penguins in a Christmas Caper (2005)
Penguins of Madagascar (2014)

Rekomendowane filmy po obejrzeniu 'Hobbit: The Desolation of Smaug, The':
Hobbit: An Unexpected Journey, The (2012)
Hobbit: The Desolation of Smaug, The (2013)
The Hobbit: The Battle of the Five Armies (2014)
